# Proyecto chatbot

## Librerias:

In [1]:
import numpy as np
import pandas as pd
import sklearn
import nltk
import re
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
import pickle

import spacy
nlp=spacy.load('es_core_news_sm')

from nltk.stem import SnowballStemmer
spanish_stemmer = SnowballStemmer('spanish')

In [2]:
respuestas = pd.read_csv('tbl_respuestas.csv', sep=',', encoding='utf_8')
respuestas.drop("Modalidad", axis=1, inplace=True)
#respuestas

In [3]:
aa = input("texto: ")

In [4]:
df_textos = pd.DataFrame(columns = ['oracion'])
df_textos.loc[0] = [aa] 
df_textos 

,oracion
0,como es la carrera de agronomia


In [50]:
def PreProcesar_carrera(Corpus, POS=False, Lema=True, Stem=True):
    
    
    # Generar una lista de documentos de spacy para tratar el POS Tagging y la Lematización
    docs=[]
    for oracion in Corpus:
        docs.append(nlp(oracion.lower())) #La lematización funciona mejor en minúsculas
    
    # Crear una lista de oraciones, donde cada elemento es una lista de palabras.
    # Cada palabra está definida por una tupla (Texto, POSTag, Lema)
    # Se omiten los tokens que son identificados como signos de puntuación
    oraciones=[]
    for doc in docs:
        oracion=[]
        for token in doc:
            if token.pos_ != 'PUNCT':
                oracion.append((token.text, token.pos_, token.lemma_))
        oraciones.append(oracion)
    
    fc = open('stopwords.txt', 'r', encoding='utf8')
    stopwords = fc.read().split('\n')
    fc.close()
    oraciones = [[palabra for palabra in oracion if palabra[2] not in stopwords] for oracion in oraciones]
    
    # Stemming
    if Stem==True:
        oraciones_aux=[]
        for oracion in oraciones:
            oracion_aux=[]
            for palabra in oracion:
                p_texto, p_pos, p_lema = palabra
                # Si Lema es True, se Stemmatiza el lema; si no, se Stemmatiza la palabra original
                if Lema==True:
                    oracion_aux.append((p_texto, p_pos, p_lema, spanish_stemmer.stem(p_lema)))
                else:
                    oracion_aux.append((p_texto, p_pos, p_lema, spanish_stemmer.stem(p_texto)))
            oraciones_aux.append(oracion_aux)
        
        oraciones = oraciones_aux
    
    Corpus_Procesado = [] #Variable de salida
    
    for doc in oraciones:
        oracion = ''
        for palabra in doc:
            if Stem == True:
                # Devolver cadena de Stemming
                oracion = oracion + palabra[3]
            else:
                if Lema == True:
                    # Devolver cadena de Lemas
                    oracion = oracion + palabra[2]
                else:
                    # Devolver cadena de palabras originales
                    oracion = oracion + palabra[0]
            
            if POS == True:
                #Concatenar POS a cada palabra
                oracion = oracion + '_' + palabra[1].lower()
            
            oracion = oracion + ' '
        
        Corpus_Procesado.append(oracion)
        
    return Corpus_Procesado

def Corregir_Documentos_carrera(df_textos, columnas, POS=False, Lema=True, Stem=True):

    for col in columnas:
        df_textos[col] = PreProcesar_carrera(list(df_textos[col]), POS, Lema, Stem)
    
    # Sanear el DataFrame eliminando los duplicados y reindexándolo
    df_textos = df_textos.drop_duplicates().reset_index(drop=True)
    
    return df_textos

#####################################################################################################

In [51]:
vari_df_textos_carr= df_textos.copy()
carrera_corr = Corregir_Documentos_carrera(vari_df_textos_carr,['oracion'],False,True,True)
carrera_corr

,oracion
0,agronomi


In [52]:
#####################################################################################################
def PreProcesar_w5(Corpus, POS=False, Lema=True, Stem=True):
    
    
    # Generar una lista de documentos de spacy para tratar el POS Tagging y la Lematización
    docs=[]
    for oracion in Corpus:
        docs.append(nlp(oracion.lower())) #La lematización funciona mejor en minúsculas
    
    # Crear una lista de oraciones, donde cada elemento es una lista de palabras.
    # Cada palabra está definida por una tupla (Texto, POSTag, Lema)
    # Se omiten los tokens que son identificados como signos de puntuación
    oraciones=[]
    for doc in docs:
        oracion=[]
        for token in doc:
            if token.pos_ != 'PUNCT':
                oracion.append((token.text, token.pos_, token.lemma_))
        oraciones.append(oracion)
    
    ww = open('stopwords_sin_w5.txt', 'r', encoding='utf8')
    stopwords = ww.read().split('\n')
    #stopwords=[x.lower() for x in stopwords]
    ww.close()
    oraciones = [[palabra for palabra in oracion if palabra[2] not in stopwords] for oracion in oraciones]
    
    # Stemming
    if Stem==True:
        oraciones_aux=[]
        for oracion in oraciones:
            oracion_aux=[]
            for palabra in oracion:
                p_texto, p_pos, p_lema = palabra
                # Si Lema es True, se Stemmatiza el lema; si no, se Stemmatiza la palabra original
                if Lema==True:
                    oracion_aux.append((p_texto, p_pos, p_lema, spanish_stemmer.stem(p_lema)))
                else:
                    oracion_aux.append((p_texto, p_pos, p_lema, spanish_stemmer.stem(p_texto)))
            oraciones_aux.append(oracion_aux)
        
        oraciones = oraciones_aux
    
    Corpus_Procesado = [] #Variable de salida
    
    for doc in oraciones:
        oracion = ''
        for palabra in doc:
            if Stem == True:
                # Devolver cadena de Stemming
                oracion = oracion + palabra[3]
            else:
                if Lema == True:
                    # Devolver cadena de Lemas
                    oracion = oracion + palabra[2]
                else:
                    # Devolver cadena de palabras originales
                    oracion = oracion + palabra[0]
            
            if POS == True:
                #Concatenar POS a cada palabra
                oracion = oracion + '_' + palabra[1].lower()
            
            oracion = oracion + ' '
        
        Corpus_Procesado.append(oracion)
        
    return Corpus_Procesado

def Corregir_Documentos_w5(df_textos, columnas, POS=False, Lema=True, Stem=True):

    for col in columnas:
        df_textos[col] = PreProcesar_w5(list(df_textos[col]), POS, Lema, Stem)
    
    # Sanear el DataFrame eliminando los duplicados y reindexándolo
    df_textos = df_textos.drop_duplicates().reset_index(drop=True)
    
    return df_textos

In [53]:
vari_df_textos_w5= df_textos.copy()
w5_corr = Corregir_Documentos_w5(vari_df_textos_w5,['oracion'],False,True,True)
w5_corr

,oracion
0,com


In [56]:
#####################################################################################################
def PreProcesar_intents(Corpus, POS=False, Lema=True, Stem=True):
    
    
    # Generar una lista de documentos de spacy para tratar el POS Tagging y la Lematización
    docs=[]
    for oracion in Corpus:
        docs.append(nlp(oracion.lower())) #La lematización funciona mejor en minúsculas
    
    # Crear una lista de oraciones, donde cada elemento es una lista de palabras.
    # Cada palabra está definida por una tupla (Texto, POSTag, Lema)
    # Se omiten los tokens que son identificados como signos de puntuación
    oraciones=[]
    for doc in docs:
        oracion=[]
        for token in doc:
            if token.pos_ != 'PUNCT':
                oracion.append((token.text, token.pos_, token.lemma_))
        oraciones.append(oracion)
    
    ww = open('stopwords_intents.txt', 'r', encoding='utf8')
    stopwords = ww.read().split('\n')
    stopwords=[x.lower() for x in stopwords]
    ww.close()
    oraciones = [[palabra for palabra in oracion if palabra[2] not in stopwords] for oracion in oraciones]
    
    # Stemming
    if Stem==True:
        oraciones_aux=[]
        for oracion in oraciones:
            oracion_aux=[]
            for palabra in oracion:
                p_texto, p_pos, p_lema = palabra
                # Si Lema es True, se Stemmatiza el lema; si no, se Stemmatiza la palabra original
                if Lema==True:
                    oracion_aux.append((p_texto, p_pos, p_lema, spanish_stemmer.stem(p_lema)))
                else:
                    oracion_aux.append((p_texto, p_pos, p_lema, spanish_stemmer.stem(p_texto)))
            oraciones_aux.append(oracion_aux)
        
        oraciones = oraciones_aux
    
    Corpus_Procesado = [] #Variable de salida
    
    for doc in oraciones:
        oracion = ''
        for palabra in doc:
            if Stem == True:
                # Devolver cadena de Stemming
                oracion = oracion + palabra[3]
            else:
                if Lema == True:
                    # Devolver cadena de Lemas
                    oracion = oracion + palabra[2]
                else:
                    # Devolver cadena de palabras originales
                    oracion = oracion + palabra[0]
            
            if POS == True:
                #Concatenar POS a cada palabra
                oracion = oracion + '_' + palabra[1].lower()
            
            oracion = oracion + ' '
        
        Corpus_Procesado.append(oracion)
        
    return Corpus_Procesado

def Corregir_Documentos_intents(df_textos, columnas, POS=False, Lema=True, Stem=True):

    for col in columnas:
        df_textos[col] = PreProcesar_intents(list(df_textos[col]), POS, Lema, Stem)
    
    # Sanear el DataFrame eliminando los duplicados y reindexándolo
    df_textos = df_textos.drop_duplicates().reset_index(drop=True)
    
    return df_textos

In [57]:
vari_df_textos_inte= df_textos.copy()
inte_corr = Corregir_Documentos_intents(vari_df_textos_inte,['oracion'],False,True,True)
inte_corr

,oracion
0,com carrer


In [58]:
vari_df_textos_sub_inte= df_textos.copy()
sub_inte_corr = Corregir_Documentos_intents(vari_df_textos_sub_inte,['oracion'],False,True,True)
sub_inte_corr

,oracion
0,com carrer


In [60]:
carrera_corr

,oracion
0,agronomi


In [68]:
vectorizador_carreras = pickle.load(open("vectorizador_carreras.pkl","rb"))
df_matriz_carreras = pickle.load(open("df_matriz_carreras.pkl","rb"))


In [101]:
#vectorizador_carreras.get_feature_names()

In [77]:
array_carreras=vectorizador_carreras.transform([carrera_corr['oracion'][0][:-1]])
#y=vectorizador.transform(['artificial cienci'])
array_carreras.toarray()


array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [78]:
modelo_carreras = pickle.load(open("modelo_carreras.sav","rb"))
#carrera = (clf_carrera.predict_proba(lola))[:,1]
carrera =sorted(list(modelo_carreras.predict_proba(array_carreras)[0]))[-1]
carrera2 = modelo_carreras.predict(array_carreras)

In [79]:
carrera

0.9999951153912673

In [80]:
carrera2

array(['Agro'], dtype=object)

In [82]:
###############################################################################
w5_corr

,oracion
0,com


In [102]:
vectorizador_w5 = pickle.load(open("vectorizador_w5.pkl","rb"))
#vectorizador_w5.get_feature_names()

In [81]:
array_w5=vectorizador_w5.transform([w5_corr['oracion'][0][:-1]])
#y=vectorizador.transform(['artificial cienci'])
array_w5.toarray()

array([[0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [85]:
modelo_w5 = pickle.load(open("modelo_w5.sav","rb"))
#carrera = (clf_carrera.predict_proba(lola))[:,1]
w5 = sorted(list(modelo_w5.predict_proba(array_w5)[0]))[-1]
w5_2 = modelo_w5.predict(array_w5)

In [86]:
w5

0.9999994085903269

In [87]:
w5_2 

array(['como'], dtype=object)

In [93]:
#################################################################################
inte_corr

,oracion
0,com carrer


In [100]:
vectorizador_intents = pickle.load(open("vectorizador_intents.pkl","rb"))
#vectorizador_intents.get_feature_names()

In [89]:
array_intents=vectorizador_intents.transform([inte_corr['oracion'][0][:-1]])
#y=vectorizador.transform(['artificial cienci'])
array_intents.toarray()

array([[0., 0., 0., ..., 0., 0., 0.]])

In [90]:
modelo_intents = pickle.load(open("modelo_intents.sav","rb"))
#carrera = (clf_carrera.predict_proba(lola))[:,1]
intents = sorted(list(modelo_intents.predict_proba(array_intents)[0]))[-1]
intents_2 = modelo_intents.predict(array_intents)

In [91]:
intents

0.9999999999984768

In [92]:
intents_2 

array(['generalidades'], dtype=object)

In [94]:
###################################################################################
sub_inte_corr

,oracion
0,com carrer


In [103]:
vectorizador_sub_intents = pickle.load(open("vectorizador_sub_intents.pkl","rb"))
#vectorizador_sub_intents.get_feature_names()

In [96]:
array_sub_intents=vectorizador_sub_intents.transform([sub_inte_corr['oracion'][0][:-1]])
#y=vectorizador.transform(['artificial cienci'])
array_sub_intents.toarray()

array([[0., 0., 0., ..., 0., 0., 0.]])

In [97]:
modelo_sub_intents = pickle.load(open("modelo_sub_intents.sav","rb"))
#carrera = (clf_carrera.predict_proba(lola))[:,1]
sub_intents = sorted(list(modelo_sub_intents.predict_proba(array_sub_intents)[0]))[-1]
sub_intents_2 = modelo_sub_intents.predict(array_sub_intents)

In [98]:
sub_intents

0.9999999189254071

In [99]:
sub_intents_2 

array(['todas'], dtype=object)

['generalidades', 'todas', 'Agro', 'como']

In [116]:
listita = [intents_2[0], sub_intents_2[0], carrera2[0], w5_2[0]]
df_textos_listo = pd.DataFrame(columns = ['Intencion',"SubIntencion","Carrera","w5"])
df_textos_listo.loc[0] = listita
df_textos_listo 

,Intencion,SubIntencion,Carrera,w5
0,generalidades,todas,Agro,como


In [ ]:
search = (df_textos_listo["Carrera"]==respuestas["Carrera"])
search

In [115]:
respuestas

,IDRespuesta,Intencion,SubIntencion,Carrera,w5,Respuesta
0,1,generalidades,todas,IA,todas,A partir de la captación y procesamiento de da...
1,2,generalidades,todas,MM,todas,"Como Diseñador de Multimedios, serás un profes..."
2,3,generalidades,todas,DG,todas,El diseño gráfico es una disciplina indispensa...
3,4,generalidades,todas,Pub,todas,"En la actualidad y desde siempre, la publicida..."
4,5,generalidades,todas,RRPP,todas,Las Relaciones Públicas constituyen un conjunt...
...,...,...,...,...,...,...
81,82,tramites,todas,todas,donde,\nLa preinscripción puede hacerse a través de ...
82,83,tramites,preinscripcion,todas,donde,\nLa preinscripción puede hacerse a través de ...
83,84,tramites,cies,todas,donde,La preinscripción puede hacerse a través de nu...
84,85,tramites,requisitos,todas,donde,La preinscripción puede hacerse a través de nu...
